# Installation
CliMETAR users some specific packages, that are not available by default. Therefore, you should download and install them before using the other scripts.

## Cartopy
Cartopy is a library for plotting maps. It also handels different projections and handles coordinate systems.

In [1]:
!conda install -qyS cartopy


# All requested packages already installed.



## Natural Earth
For plotting maps, the Natural Earth dataset is nessesary. Please download the nessesary files, and store them in the `CliMETAR/natural_earth` folder:
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip]()
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip]()
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/NE1_LR_LC_SR_W_DR.zip]()

## Station data
Station data is provided in the repository, but should be regularly updated. Use the script below (on a machine with internet access) to regenerate a new file.
Store a new file in the `CliMetar/resources` folder

```{python}
    pass
```